In [ ]:
import os 
os.chdir(r'Q:\sachuriga\Sachuriga_Python/quattrocolo-nwb4fp\src')

from neurochat.nc_data import NData
from neurochat.nc_spike import NSpike
from neurochat.nc_spatial import NSpatial
import neurochat.nc_plot as nc_plot
from neurochat.nc_lfp import NLfp
import matplotlib.pyplot as plt
import numpy as np
from pynwb import NWBHDF5IO
import matplotlib.pyplot as plt
import numpy as np
import math
import pynapple as nap
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import normalize

import sys
import nwb4fp.analyses.maps as mapp
from nwb4fp.analyses.examples.tracking_plot import plot_ratemap_ax,plot_path
from nwb4fp.analyses.fields import separate_fields_by_laplace, separate_fields_by_dilation,find_peaks,separate_fields_by_laplace_of_gaussian,calculate_field_centers,distance_to_edge_function, remove_fields_by_area, map_pass_to_unit_circle,which_field,compute_crossings
from elephant.statistics import time_histogram, instantaneous_rate
from nwb4fp.analyses import maps
from nwb4fp.analyses.data import pos2speed,speed_filtered_spikes,load_speed_fromNWB,load_units_fromNWB,get_filed_num,Speed_filtered_spikes
from nwb4fp.data.helpers import unit_location_ch
from scipy.ndimage import gaussian_filter
import ast
import pandas as pd

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)
np.set_printoptions(threshold=np.inf)
## unit11: unit_num = 8, ch = 13
filepath = r"S:\Sachuriga/nwb/test4neo/65165_2023-07-10_15-08-59_A_phy_k_manual.nwb"
npdata = nap.load_file(filepath)
npdata

## Load data
pos_cord = load_speed_fromNWB(npdata['XY_mid_brain'])

## filter speed
raw_pos,combined_array, mask,speeds,smoothed_speed,filtered_speed = pos2speed(pos_cord[:,0], # times
                            pos_cord[:,1], # x
                            pos_cord[:,2], # y
                            filter_speed=True, 
                            min_speed = 0.1)

## filter spikes with speed
unit_num = 13
raw_pos=combined_array
# ## filter spikes with speed
# spk = speed_filtered_spikes(spikes_time,
#                             pos_cord[:,0], # times
#                             mask)
#for i in range(40):
spikes_time = load_units_fromNWB(npdata['units'], unit_num = unit_num)
spikes_time = Speed_filtered_spikes(spikes_time,
                            pos_cord[:,0],mask)

spk = speed_filtered_spikes(spikes_time,
                            pos_cord[:,0])

plot_ratemap_ax(raw_pos[:,1], # x
            raw_pos[:,2], # y
            raw_pos[:,0], # times
            spikes_time,
            box_size=[1.0, 1.0], 
            bin_size=0.05,
            smoothing=0.1)


x_input = npdata['units']['x'][unit_num]
y_input = npdata['units']['y'][unit_num]

In [ ]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt

def calculate_spike_theta_modulation(spike_times, fs=1000, freq_range=(0, 125), theta_range=(6, 12)):
    """
    Calculate theta modulation index for a spike train and return PSD and full ACG for plotting.
    
    Parameters:
    -----------
    spike_times : array-like
        Array of spike timestamps in seconds
    fs : float
        Sampling frequency in Hz (default: 1000 Hz)
    freq_range : tuple
        Frequency range for total power calculation (default: 0-125 Hz)
    theta_range : tuple
        Frequency range for theta band (default: 6-12 Hz)
        
    Returns:
    --------
    bool : Whether the spike train is theta modulated (index > 5)
    float : Theta modulation index
    array : Frequencies from PSD
    array : Power spectral density values
    array : Time lags for ACG
    array : Full autocorrelogram values (including negative lags)
    """
    if len(spike_times) > 18000:
        fs = 10000

    # Convert spike times to binary spike train
    spike_times = np.array(spike_times)
    duration = spike_times[-1] - spike_times[0]
    n_samples = int(duration * fs) + 1
    spike_train = np.zeros(n_samples)
    spike_indices = ((spike_times - spike_times[0]) * fs).astype(int)
    spike_train[spike_indices] = 1
    
    # Calculate full autocorrelogram
    autocorr_full = signal.correlate(spike_train, spike_train, mode='full')
    center = len(autocorr_full) // 2
    autocorr_full[center] = 0  # Exclude zero-lag
    lags = np.arange(-center, center + 1) / fs  # Time lags in seconds
    
    # Take positive lags for PSD calculation
    autocorr_positive = autocorr_full[center:]  # Positive lags only
    
    # Calculate power spectral density
    freqs, psd = signal.welch(autocorr_positive, fs=fs, nperseg=min(1024, len(autocorr_positive)))
    
    # Find indices for frequency ranges
    freq_mask_total = (freqs >= freq_range[0]) & (freqs <= freq_range[1])
    freq_mask_theta = (freqs >= theta_range[0]) & (freqs <= theta_range[1])
    
    # Find peak theta frequency
    theta_psd = psd[freq_mask_theta]
    theta_freqs = freqs[freq_mask_theta]
    peak_theta_idx = np.argmax(theta_psd)
    peak_theta_freq = theta_freqs[peak_theta_idx]
    
    # Calculate power in 2 Hz window around peak theta frequency
    theta_window_mask = (freqs >= (peak_theta_freq - 1)) & (freqs <= (peak_theta_freq + 1))
    
    # Calculate power ratios
    total_power = np.sum(psd[freq_mask_total])
    theta_power = np.sum(psd[theta_window_mask])
    
    # Calculate modulation index
    modulation_index = (theta_power / total_power) * 100
    
    # Check if theta modulated (index > 5)
    is_theta_modulated = modulation_index > 5
    
    return is_theta_modulated, modulation_index, freqs, psd, lags, autocorr_full

def plot_acg_and_psd(lags, autocorr, freqs, psd, theta_range=(6, 12)):
    """
    Plot the autocorrelogram (ACG) and power spectral density (PSD) with theta range highlighted.
    """
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
    
    # Plot ACG
    ax1.plot(lags, autocorr, 'b-', label='Autocorrelogram')
    ax1.set_xlabel('Time Lag (s)')
    ax1.set_ylabel('Correlation')
    ax1.set_title('Spike Train Autocorrelogram (Zero-lag Excluded)')
    ax1.grid(True)
    ax1.set_xlim(-0.5, 0.5)  # Show ±0.5 seconds to include negative lags
    ax1.legend()
    
    # Plot PSD
    ax2.plot(freqs, psd, 'b-', label='PSD')
    ax2.axvspan(theta_range[0], theta_range[1], color='gray', alpha=0.2, label='Theta range (6-12 Hz)')
    ax2.set_xlabel('Frequency (Hz)')
    ax2.set_ylabel('Power Spectral Density')
    ax2.set_title('Power Spectral Density')
    ax2.grid(True)
    ax2.set_xlim(0, 50)  # Limit to 50 Hz for theta visualization
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

# Example usage with synthetic spike times
#spike_times = np.sort(np.random.uniform(0, 10, 100))  # 100 random spikes over 10 seconds
is_modulated, mod_index, freqs, psd, lags, autocorr = calculate_spike_theta_modulation(spikes_time)

print(f"Theta modulation index: {mod_index:.2f}")
print(f"Is theta modulated: {is_modulated}")

# Plot both ACG and PSD
plot_acg_and_psd(lags, autocorr, freqs, psd)

In [ ]:
import numpy as np
from scipy import signal
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

def calculate_spike_theta_modulation(spike_times, fs=1000, freq_range=(0, 125), theta_range=(6, 12)):
    """
    Calculate theta modulation index for a spike train and return PSD and ACG for plotting.
    Excludes zero-lag from autocorrelation.
    
    Parameters:
    -----------
    spike_times : array-like
        Array of spike timestamps in seconds
    fs : float
        Sampling frequency in Hz (default: 1000 Hz)
    freq_range : tuple
        Frequency range for total power calculation (default: 0-125 Hz)
    theta_range : tuple
        Frequency range for theta band (default: 6-12 Hz)
        
    Returns:
    --------
    bool : Whether the spike train is theta modulated (index > 5)
    float : Theta modulation index
    array : Frequencies from PSD
    array : Power spectral density values
    array : Time lags for ACG
    array : Autocorrelogram values
    """
    #spike_train=spikes_time
    # Convert spike times to binary spike train
    
    duration = spike_times[-1] - spike_times[0]
    n_samples = int(duration * fs) + 1
    spike_train = np.zeros(n_samples)
    spike_indices = ((spike_times - spike_times[0]) * fs).astype(int)
    spike_train[spike_indices] = 1
    
    ##Calculate autocorrelogram (full mode)
    autocorr = signal.correlate(spike_train, spike_train, mode='full')
    
    # Exclude the zero-lag point (center of the autocorrelation)
    center = len(autocorr) // 2
    autocorr[center] = 0  # Set the zero-lag to 0
    autocorr = gaussian_filter1d(autocorr, sigma=2)
    # Take only positive lags for PSD calculation
    autocorr_psd = autocorr[center + 1:]  # Start from t=1
    
    # Calculate power spectral density
    freqs, psd = signal.welch(autocorr_psd, fs=fs, nperseg=min(1024, len(autocorr_psd)))
    
    # Find indices for frequency ranges
    freq_mask_total = (freqs >= freq_range[0]) & (freqs <= freq_range[1])
    freq_mask_theta = (freqs >= theta_range[0]) & (freqs <= theta_range[1])
    
    # Find peak theta frequency
    theta_psd = psd[freq_mask_theta]
    theta_freqs = freqs[freq_mask_theta]
    peak_theta_idx = np.argmax(theta_psd)
    peak_theta_freq = theta_freqs[peak_theta_idx]
    
    # Calculate power in 2 Hz window around peak theta frequency
    theta_window_mask = (freqs >= (peak_theta_freq - 1)) & (freqs <= (peak_theta_freq + 1))
    
    # Calculate power ratios
    total_power = np.sum(psd[freq_mask_total])
    theta_power = np.sum(psd[theta_window_mask])
    
    # Calculate modulation index
    modulation_index = (theta_power / total_power) * 100
    
    # Check if theta modulated (index > 5)
    is_theta_modulated = modulation_index > 5
    
    # Prepare ACG for plotting (full range with positive and negative lags)
    autocorr_full = signal.correlate(spike_train, spike_train, mode='full')
    autocorr_full[center] = 0  # Exclude zero-lag
    lags = np.arange(-center, center + 1) / fs  # Time lags in seconds
    
    return is_theta_modulated, modulation_index, freqs, psd, lags, autocorr_full

def plot_acg_and_psd(lags, autocorr, freqs, psd, theta_range=(6, 12)):
    """
    Plot the autocorrelogram (ACG) and power spectral density (PSD) with theta range highlighted.
    """
    # Create subplots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8))
    
    # Plot ACG
    ax1.plot(lags, autocorr, 'b-', label='Autocorrelogram')
    ax1.set_xlabel('Time Lag (s)')
    ax1.set_ylabel('Correlation')
    ax1.set_title('Spike Train Autocorrelogram (Zero-lag Excluded)')
    ax1.grid(True)
    ax1.set_xlim(-0.5, 0.5)  # Limit to ±0.5 seconds for better visualization
    ax1.legend()
    
    # Plot PSD
    ax2.plot(freqs, psd, 'b-', label='PSD')
    ax2.axvspan(theta_range[0], theta_range[1], color='gray', alpha=0.2, label='Theta range (6-12 Hz)')
    ax2.set_xlabel('Frequency (Hz)')
    ax2.set_ylabel('Power Spectral Density')
    ax2.set_title('Power Spectral Density')
    ax2.grid(True)
    ax2.set_xlim(0, 50)  # Limit to 50 Hz for better visualization of theta range
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

# Example usage with synthetic spike times

#spikes_time = np.sort(np.random.uniform(0, 10, 100))  # 100 random spikes over 10 seconds
is_modulated, mod_index, freqs, psd, lags, autocorr = calculate_spike_theta_modulation(spikes_time)

print(f"Theta modulation index: {mod_index:.2f}")
print(f"Is theta modulated: {is_modulated}")

# Plot both ACG and PSD
plot_acg_and_psd(lags, autocorr, freqs, psd)

In [ ]:
import nwb4fp.analyses.maps as mapp
ch_num = unit_location_ch(x=x_input,y=y_input)
## get rate_maps
maps = mapp.SpatialMap(box_size=[1.0, 1.0], 
                       bin_size=0.05,
                       smoothing=0.1)

## generate ratemaps
rate_map = maps.rate_map(raw_pos[:,1], 
                        raw_pos[:,2], 
                        raw_pos[:,0], 
                        spikes_time)

## get_fileds with center of the files
fields = separate_fields_by_laplace_of_gaussian(rate_map, 
                                    minimum_field_area=9)
fiesld_afremoval = remove_fields_by_area(fields, minimum_field_area=9)
bc = calculate_field_centers(rate_map, fiesld_afremoval, center_method='center_of_mass')
v = get_filed_num(fiesld_afremoval)

## get the center of the fields
for field_nums in v:
    y_c = bc[field_nums-1][0]
    x_c = bc[field_nums-1][1]

In [ ]:
## crosssings
f = which_field(raw_pos[:,1],raw_pos[:,2],fiesld_afremoval,[1.0,1.0])

in_field = np.where(f==1)[0]
indices = np.zeros(len(raw_pos[:,1]))
indices[in_field]=1

en,ex = compute_crossings(indices)
filter_loger_runs = ex-en
filtered_enter = en[filter_loger_runs>100]
filtered_exit = ex[filter_loger_runs>100]


restrcir_en=[]
restrcir_ex=[]

for i in range(len(filtered_enter)):
    if np.mean(smoothed_speed[filtered_enter[i]:filtered_exit[i]]) >= 0.05:
        restrcir_en.append(filtered_enter[i])
        restrcir_ex.append(filtered_exit[i])

## infields cords
pos_cord=raw_pos
xf = pos_cord[:,1][in_field]
yf = pos_cord[:,2][in_field]
tf = pos_cord[:,0][in_field]#
spk_in = spk[in_field-1]

#Exatract LFP
lfp_times = npdata['lfp_times']
lfp = npdata['lfp_raw']

In [ ]:
count = np.sum(fiesld_afremoval  == 1)
radius =  np.sqrt((count/400)*2500 / np.pi)
radius 

In [ ]:
count/

In [ ]:
fiesld_afremoval

In [ ]:
import numpy as np
from scipy import signal
from scipy.signal import hilbert
import matplotlib.pyplot as plt

fs = 1000
# 1. 带通滤波 (6-11 Hz)
def butter_bandpass(lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='band')
    return b, a

def bandpass_filter(data, lowcut, highcut, fs, order=4):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = signal.filtfilt(b, a, data)  # 前后向滤波，避免相位移
    return y

lfp_filtered = bandpass_filter(lfp[:,ch_num], 5, 12, fs)

# 2. 使用希尔伯特变换计算瞬时相位
analytic_signal = hilbert(lfp_filtered)

# 计算相位（弧度）
phase_rad = np.angle(analytic_signal)

# 展开相位，避免-π到π的跳跃
phase_rad_unwrapped = np.unwrap(phase_rad)

# 转换为角度
phase_deg = phase_rad_unwrapped * (180 / np.pi)

# 调整到0°到720°范围（取决于信号长度和周期）
# 如果需要限制在0°到720°，可以用模运算或其他逻辑
theta_phase = phase_deg % 360 # 取模720，或者根据需求自定义

In [ ]:
x = pos_cord[:,1]
y = pos_cord[:,2]
t = pos_cord[:,0]

fig, ax3 = plt.subplots()

fig, ax2 = plt.subplots()
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
full_phase=[]
full_x=[]
full_r=[]
full_theta = []


for slide_s,slide_stop in zip(restrcir_en, restrcir_ex):
    #plot_path(x[slide_s:slide_stop],y[slide_s:slide_stop],t[slide_s:slide_stop],box_size=1,spike_times = spikes_time, ax=ax3)
    r, theta, pdcd, pdmd = map_pass_to_unit_circle(x[slide_s:slide_stop],y[slide_s:slide_stop],t[slide_s:slide_stop], x_c/20, y_c/20, field=fiesld_afremoval, box_size=[1.0, 1.0])
    phase_value=[]

    temp_time = t[slide_s:slide_stop]
    temp_x = x[slide_s:slide_stop]
    temp_y = y[slide_s:slide_stop]
    temp = spikes_time[spikes_time>t[slide_s]]
    temp1 = temp[temp<=t[slide_stop-1]]
    phase_value=[]
    phase_temp = []
    pdcd_temp = []
    r_temp = []



    if len(theta)>100:
        ax.plot(theta, r, label='Example Curve', color='black', linewidth=1)
        ax2.plot(y[slide_s:slide_stop], x[slide_s:slide_stop], color='black', linewidth=1)

    
    if len(temp1)>0:
        for t1 in temp1:
            index_run = np.searchsorted(temp_time, t1)
            index_theta = np.searchsorted(lfp_times, t1)
            phase_value.append(theta_phase[index_theta])
            phase_temp.append(theta[index_run])
            r_temp.append(r[index_run])

            if len(theta)>100:
                ax.scatter(theta[index_run], r[index_run], color='red', marker='*')
                ax2.scatter(temp_y[index_run], temp_x[index_run], color='red', marker='*')

            pdcd_temp.append(pdcd[index_run])

            full_r.append(r[index_run])
            full_phase.append(theta_phase[index_theta])
            full_x.append(pdcd[index_run])
            full_theta.append(theta[index_run])
        ax3.scatter(pdcd_temp,phase_value)
        
    ax2.axis('equal')
    ax2.set_xlim(0, 1)  # Adjust based on pdcd range
    ax2.set_ylim(0, 1) 
    ax2.invert_yaxis()


In [ ]:
x = pos_cord[:,1]
y = pos_cord[:,2]
t = pos_cord[:,0]

fig, ax3 = plt.subplots()

fig, ax2 = plt.subplots()
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
full_phase=[]
full_x=[]
full_r=[]
full_theta = []


for slide_s,slide_stop in zip(restrcir_en, restrcir_ex):
    #plot_path(x[slide_s:slide_stop],y[slide_s:slide_stop],t[slide_s:slide_stop],box_size=1,spike_times = spikes_time, ax=ax3)
    r, theta, pdcd, pdmd = map_pass_to_unit_circle(x[slide_s:slide_stop],y[slide_s:slide_stop],t[slide_s:slide_stop], x_c/20, y_c/20, field=fiesld_afremoval, box_size=[1.0, 1.0])
    phase_value=[]

    temp_time = t[slide_s:slide_stop]
    temp = spikes_time[spikes_time>t[slide_s]]
    temp1 = temp[temp<=t[slide_stop-1]]
    phase_value=[]
    phase_temp = []
    pdcd_temp = []
    r_temp = []



    if len(theta)>100:
        ax.plot(theta, r, label='Example Curve', color='black', linewidth=1)
        ax2.plot(x[slide_s:slide_stop], y[slide_s:slide_stop], color='black', linewidth=1)
        ax2.set_xlim([0,1])
    if len(temp1)>0:
        for t1 in temp1:
            index_run = np.searchsorted(temp_time, t1)
            index_theta = np.searchsorted(lfp_times, t1)
            phase_value.append(theta_phase[index_theta])
            phase_temp.append(theta[index_run])
            r_temp.append(r[index_run])
            if len(theta)>100:
                ax.scatter(theta[index_run], r[index_run], color='red', marker='*')
            pdcd_temp.append(pdcd[index_run])

            full_r.append(r[index_run])
            full_phase.append(theta_phase[index_theta])
            full_x.append(pdcd[index_run])
            full_theta.append(theta[index_run])
        ax3.scatter(pdcd_temp,phase_value)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d


# Step 1: Define your list of spike phases in degrees (as floats)
# Replace this with your actual data
#full_phase = [0.0, 90.5, 180.2, 270.8, 360.0, 450.3, 540.7]  # Example list of phases in degrees (floats)
# If you have your own data, use:
# full_phase = your_spike_phases_in_degrees

# Convert the list to a NumPy array for element-wise operations
spike_phase = np.array(full_phase)

# Step 2: 为了循环周期性，将相位限制在 [0, 360) 范围内
# Step 2: 计算直方图
bins = 144  # 分成 20 个区间，每区间 18 度
hist, bin_edges = np.histogram(spike_phase, bins=bins, range=(0, 360))
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2  # 每个区间的中心点

# Step 3: 应用高斯平滑，sigma = 45 度
# 因为 bin 的宽度是 18 度，sigma = 45 度需要换算成 bin 单位
sigma_in_bins = 5/ (360 / bins)  # 45 度除以每个 bin 的宽度
smoothed_hist = gaussian_filter1d(hist, sigma=sigma_in_bins, mode='wrap')  # 'wrap' 处理周期性边界
preferred_phase = bin_centers[np.argmax(smoothed_hist)]
# Step 4: 绘制平滑后的直方图
plt.figure(figsize=(8, 6))
plt.bar(bin_centers, smoothed_hist, width=360/bins, color='blue', label='Smoothed (Gaussian, σ=45°)')
plt.bar(bin_centers, hist, width=360/bins, color='skyblue', edgecolor='black', alpha=0.5, label='Original Histogram')

# 自定义图表
plt.title("Histogram of Spike Phases with Gaussian Smoothing")
plt.xlabel("Phase (degrees)")
plt.ylabel("Count")
plt.legend()
plt.grid(True, alpha=0.3)

# Step 5: 显示图表
plt.show()
preferred_phase

min_phase = bin_centers[np.argmin(smoothed_hist)]

In [ ]:
import numpy as np
from scipy.ndimage import gaussian_filter1d

# 步骤 1：将尖峰分到1°的bin中
bins = np.arange(0, 360, 1)  # 360个bin，从0°到359°
hist, bin_edges = np.histogram(full_phase, bins=bins, range=(0, 360))
sigma = 45  # 标准差45°
hist_smoothed = gaussian_filter1d(hist, sigma=sigma)  # 平滑

# 步骤 3：找到最小尖峰数的相位
min_phase_idx = np.argmin(hist_smoothed)  # 最小值的索引
phase_zero = bins[min_phase_idx]  # 定义为相位零点
print(f"Phase zero (minimum firing phase): {phase_zero}°")

# 可选：调整相位（如果需要）
# 如果想把所有相位相对于 phase_zero 重新定义
adjusted_phases = (spike_phase + phase_zero) % 360

degrees_list = adjusted_phases
radians_list = []

for deg in degrees_list:
    radians_list.append(math.radians(deg))

fig, ax4 = plt.subplots()
ax4.scatter(full_x,degrees_list ,c='k', alpha=0.5,s=10)
ax4.scatter(full_x, degrees_list + 360,c='k', alpha=0.5,s=10)
plt.xlim(-1, 1) 
            


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# 计算二维直方图
#x_bins = np.linspace(min(lin_x), max(lin_x), 50)  # x轴分成50个bin
#y_bins = np.linspace(0, 360, 50)  # 相位分成50个bin（0°-360°）
full_x_stacked = np.hstack([full_x, full_x])
full_adjusted_phases = np.hstack([adjusted_phases, adjusted_phases+360])
heatmap, xedges, yedges = np.histogram2d(full_x_stacked,full_adjusted_phases, bins=[500, 500])

# 可选：平滑热图（可选，去掉注释使用）
from scipy.ndimage import gaussian_filter
heatmap = gaussian_filter(heatmap, sigma=40)

# 绘制热图
plt.figure(figsize=(10, 6))
plt.pcolormesh(xedges, yedges, heatmap.T, cmap='plasma')  # heatmap.T转置以匹配方向
plt.colorbar(label='Density')  # 添加颜色条表示密度
plt.xlabel('Linear Variable (e.g., Time)')
plt.ylabel('Theta Phase (degrees)')
plt.title('Phase Density Heatmap')
plt.xlim(-1, 1) 
plt.show()

In [ ]:
import numpy as np
from scipy.stats import circmean, circvar

def circular_linear_fit(theta, x, slope_range):
    """
    Perform circular-linear analysis to quantify theta phase precession.
    
    Parameters:
    - theta: Array of instantaneous theta phases (in radians) for each spike.
    - x: Array of spatial positions of the animal at spike times.
    - slope_range: Array of slopes to test (in cycles/cm).
    
    Returns:
    - best_slope: Slope that maximizes the resultant length (R).
    - max_R: Maximum resultant length indicating model fit.
    - theta_0: Phase offset for the best fit.
    """
    def calculate_residuals(theta, x, slope, theta_0):
        """Calculate angular residuals for a given slope and phase offset."""
        theta_pred = (theta_0 + 2 * np.pi * slope * x) % (2 * np.pi)  # Predicted phase
        residuals = (theta - theta_pred) % (2 * np.pi)
        residuals = np.where(residuals > np.pi, residuals - 2 * np.pi, residuals)  # Wrap to [-π, π]
        return residuals

    def resultant_length(residuals):
        """Calculate resultant length (R) of angular residuals."""
        cos_sum = np.sum(np.cos(residuals))
        sin_sum = np.sum(np.sin(residuals))
        R = np.sqrt(cos_sum**2 + sin_sum**2) / len(residuals)
        return R

    best_R = -np.inf
    best_slope = None
    best_theta_0 = None

    # Test each slope in the provided range
    for slope in slope_range:
        # Estimate theta_0 using circular mean of residuals
        theta_pred = (2 * np.pi * slope * x) % (2 * np.pi)
        residuals = (theta - theta_pred) % (2 * np.pi)
        theta_0 = circmean(residuals, high=2*np.pi, low=0)
        
        # Calculate residuals and resultant length
        residuals = calculate_residuals(theta, x, slope, theta_0)
        R = resultant_length(residuals)
        
        # Update best fit if R is larger
        if R > best_R:
            best_R = R
            best_slope = slope
            best_theta_0 = theta_0

    return best_slope, best_R, best_theta_0

def permutation_test(theta, x, best_slope, best_R, n_permutations=1000):
    """
    Perform permutation test to estimate statistical significance of the fit.
    
    Parameters:
    - theta: Array of theta phases.
    - x: Array of spatial positions.
    - best_slope: Slope from the original fit.
    - best_R: Resultant length from the original fit.
    - n_permutations: Number of permutations for the test.
    
    Returns:
    - p_value: P-value indicating significance of the fit.
    """
    perm_R = []
    for _ in range(n_permutations):
        # Randomly permute theta values
        theta_perm = np.random.permutation(theta)
        slope, R, _ = circular_linear_fit(theta_perm, x, [best_slope])
        perm_R.append(R)
    
    # Calculate p-value as proportion of permutations with R >= best_R
    p_value = np.sum(np.array(perm_R) >= best_R) / n_permutations
    return p_value

In [ ]:
slope_range = np.tan(np.linspace(-0.1, -0.005, 1000))

full_x_stacked1 = full_x_stacked * radius
full_x_stacked2 = full_x_stacked1+radius
radians_list=[]
first_idx = (full_x_stacked <= 0) & (full_x_stacked >= -1)
secound_idx = (full_x_stacked > 0) & (full_x_stacked <= 1)


for deg in full_adjusted_phases:
    radians_list.append(math.radians(deg))


best_slope, max_R, theta_0 = circular_linear_fit(full_x_stacked1, np.array(radians_list), slope_range)  
p_value = permutation_test(full_x_stacked, np.array(radians_list), best_slope, max_R, n_permutations=1000)

print(fr"p val:{p_value}")
print(fr"slope:{best_slope}")


In [ ]:
radians_list=[]
first_idx = (full_x_stacked <= 0) & (full_x_stacked >= -1)
secound_idx = (full_x_stacked > 0) & (full_x_stacked <= 1)

full_x_stacked1 = full_x_stacked 
for deg in full_adjusted_phases:
    radians_list.append(math.radians(deg))
    
from nwb4fp.analyses.phase_precession import cl_corr
circ_lin_corr, ci, slope, phi0, RR = cl_corr(full_x_stacked1, np.array(radians_list), -.5, -0.025, 0.05, 1000, return_pval=True)
if ci <0.05:
    print("yes")
    print(fr"{ci}")
    print(fr"slope:{slope}")
    print(circ_lin_corr)

circ_lin_corr, p1, slope1, phi1, RR = cl_corr(full_x_stacked1[first_idx], np.array(radians_list)[first_idx], -0.5, -0.025, 0.05, 1000, return_pval=True)
print(fr"p val:{ci}")
print(fr"slope:{slope}")
circ_lin_corr, p2, slope2, phi2, RR = cl_corr(full_x_stacked1[secound_idx], np.array(radians_list)[secound_idx], -0.5, -0.025, 0.05, 1000, return_pval=True)
print(fr"p val:{ci}")
print(fr"slope:{slope}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 已知输出值


# 将 phi0 从弧度转换为度
phi0_deg = np.degrees(phi0)  +158.75 # 转换为度
# 如果 slope 是弧度/单位线性变量，也需要转换为度/单位线性变量
slope_deg = 360 * slope  # 假设 slope 是弧度/单位，若不是则跳过此步

# 定义线性变量范围（假设与之前一致，从 -1 到 1）
x = np.linspace(-1, 1, 50)

y1 = slope_deg * (x + 1) + phi0_deg          # 第一条线
y2 = slope_deg * (x + 1) + (phi0_deg + 360)  # 第二条线 (phi0 + 360°)
y3 = slope_deg * (x + 1) + (phi0_deg + 720)  # 第三条线 (phi0 + 720°)

# 绘制图形
plt.figure(figsize=(8, 6))
plt.scatter(full_x,degrees_list ,c='k', alpha=0.5,s=10)
plt.scatter(full_x, degrees_list + 360,c='k', alpha=0.5,s=10)

plt.plot(x, y1, 'b-', label=f'phi0 = {phi0_deg:.2f}°')
plt.plot(x, y2, 'b-', label=f'phi0 + 360 = {phi0_deg + 360:.2f}°')
plt.plot(x, y3, 'b-', label=f'phi0 + 720 = {phi0_deg + 720:.2f}°')

# 设置 y 轴范围为 0 到 720 度
plt.ylim(0, 720)

# 添加标签和标题
plt.xlabel('Position')
plt.ylabel('Theta Phase (degrees)')
plt.title('Phase Precession Slopes')

plt.grid(True)

# 显示图形
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 已知输出值

phase_zero=109
# 将 phi0 从弧度转换为度
phi0_deg = np.degrees(phi0)  + min_phase  # 转换为度
# 如果 slope 是弧度/单位线性变量，也需要转换为度/单位线性变量
slope_deg = 360 * slope  # 假设 slope 是弧度/单位，若不是则跳过此步

# 定义线性变量范围（假设与之前一致，从 -1 到 1）
x = np.linspace(-1, 1, 50)

y1 = slope_deg * (x + 1) + phi0_deg          # 第一条线
y2 = slope_deg * (x + 1) + (phi0_deg + 360)  # 第二条线 (phi0 + 360°)
y3 = slope_deg * (x + 1) + (phi0_deg + 720)  # 第三条线 (phi0 + 720°)

# 绘制图形
plt.figure(figsize=(8, 6))
plt.scatter(full_x,degrees_list ,c='k', alpha=0.5,s=10)
plt.scatter(full_x, degrees_list + 360,c='k', alpha=0.5,s=10)

plt.plot(x, y1, 'r-', label=f'phi0 = {phi0_deg:.2f}°')
plt.plot(x, y2, 'r-', label=f'phi0 + 360 = {phi0_deg + 360:.2f}°')
plt.plot(x, y3, 'r-', label=f'phi0 + 720 = {phi0_deg + 720:.2f}°')

# 定义线性变量范围（假设与之前一致，从 -1 到 1）
x = np.linspace(-1, 0, 50)
phi0_deg  = np.degrees(phi1)  + min_phase # 转换为度
# 如果 slope 是弧度/单位线性变量，也需要转换为度/单位线性变量
slope_deg = 360 * slope1  # 假设 slope 是弧度/单位，若不是则跳过此步

y1 = slope_deg * (x + 1) + phi0_deg          # 第一条线
y2 = slope_deg * (x + 1) + (phi0_deg + 360)  # 第二条线 (phi0 + 360°)
y3 = slope_deg * (x + 1) + (phi0_deg + 720)  # 第三条线 (phi0 + 720°)

plt.plot(x, y1, 'b-', label=f'phi0 = {phi0_deg:.2f}°')
plt.plot(x, y2, 'b-', label=f'phi0 + 360 = {phi0_deg + 360:.2f}°')
plt.plot(x, y3, 'b-', label=f'phi0 + 720 = {phi0_deg + 720:.2f}°')


# 定义线性变量范围（假设与之前一致，从 -1 到 1）
x = np.linspace(0, 1, 50)
phi0_deg  = y1[-1]   # 转换为度
# 如果 slope 是弧度/单位线性变量，也需要转换为度/单位线性变量
slope_deg = 360 * slope2  # 假设 slope 是弧度/单位，若不是则跳过此步

y1 = slope_deg * (x ) + phi0_deg          # 第一条线
y2 = slope_deg * (x) + (phi0_deg + 360)  # 第二条线 (phi0 + 360°)
y3 = slope_deg * (x) + (phi0_deg + 720)  # 第三条线 (phi0 + 720°)

plt.plot(x, y1, 'g-', label=f'phi0 = {phi0_deg:.2f}°')
plt.plot(x, y2, 'g-', label=f'phi0 + 360 = {phi0_deg + 360:.2f}°')
plt.plot(x, y3, 'g-', label=f'phi0 + 720 = {phi0_deg + 720:.2f}°')




# 设置 y 轴范围为 0 到 720 度
plt.ylim(0, 720)

# 添加标签和标题
plt.xlabel('Linear Variable (e.g., Time)')
plt.ylabel('Theta Phase (degrees)')
plt.title('Phase Precession Slopes')

plt.grid(True)

# 显示图形
plt.show()